In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [ ]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [18]:
X_test.shape

(2000, 13)

In [19]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [20]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [21]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 4s 10ms/step - loss: 0.5194 - accuracy: 0.7845
Epoch 2/100
250/250 [==============================] - 2s 9ms/step - loss: 0.4660 - accuracy: 0.7994
Epoch 3/100
250/250 [==============================] - 2s 10ms/step - loss: 0.4563 - accuracy: 0.8036
Epoch 4/100
250/250 [==============================] - 3s 12ms/step - loss: 0.4461 - accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4359 - accuracy: 0.8085
Epoch 6/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4263 - accuracy: 0.8141
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 0.4180 - accuracy: 0.8165
Epoch 8/100
250/250 [==============================] - 3s 11ms/step - loss: 0.4116 - accuracy: 0.8200
Epoch 9/100
250/250 [==============================] - 3s 11ms/step - loss: 0.4040 - accuracy: 0.8226
Epoch 10/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3976

250/250 [==============================] - 3s 11ms/step - loss: 0.3360 - accuracy: 0.8626
Epoch 81/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3359 - accuracy: 0.8630
Epoch 82/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3361 - accuracy: 0.8631
Epoch 83/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3347 - accuracy: 0.8654
Epoch 84/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3345 - accuracy: 0.8606
Epoch 85/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3365 - accuracy: 0.8621
Epoch 86/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3343 - accuracy: 0.8601
Epoch 87/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3356 - accuracy: 0.8631
Epoch 88/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3337 - accuracy: 0.8641
Epoch 89/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3335 - accura

# The f1 score of the class 1 score is so low due to the imbalanced Dataset.
## Trying to fix the imbalanced dataset using different algorithms

# 1) Undersampling

In [22]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [23]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [24]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [25]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [26]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 2s 11ms/step - loss: 0.6839 - accuracy: 0.5631
Epoch 2/100
102/102 [==============================] - 1s 11ms/step - loss: 0.6477 - accuracy: 0.6419
Epoch 3/100
102/102 [==============================] - 1s 12ms/step - loss: 0.6254 - accuracy: 0.6563
Epoch 4/100
102/102 [==============================] - 1s 12ms/step - loss: 0.6135 - accuracy: 0.6726
Epoch 5/100
102/102 [==============================] - 1s 14ms/step - loss: 0.6024 - accuracy: 0.6824
Epoch 6/100
102/102 [==============================] - 1s 14ms/step - loss: 0.5925 - accuracy: 0.6981
Epoch 7/100
102/102 [==============================] - 1s 12ms/step - loss: 0.5814 - accuracy: 0.6990
Epoch 8/100
102/102 [==============================] - 1s 12ms/step - loss: 0.5733 - accuracy: 0.7088
Epoch 9/100
102/102 [==============================] - 1s 13ms/step - loss: 0.5664 - accuracy: 0.7217
Epoch 10/100
102/102 [==============================] - 1s 13ms/step - loss: 0.558

102/102 [==============================] - 1s 12ms/step - loss: 0.4449 - accuracy: 0.7874
Epoch 81/100
102/102 [==============================] - 1s 11ms/step - loss: 0.4444 - accuracy: 0.7867
Epoch 82/100
102/102 [==============================] - 1s 11ms/step - loss: 0.4459 - accuracy: 0.7831
Epoch 83/100
102/102 [==============================] - 1s 13ms/step - loss: 0.4456 - accuracy: 0.7812
Epoch 84/100
102/102 [==============================] - 1s 13ms/step - loss: 0.4447 - accuracy: 0.7861
Epoch 85/100
102/102 [==============================] - 1s 13ms/step - loss: 0.4452 - accuracy: 0.7821
Epoch 86/100
102/102 [==============================] - 1s 11ms/step - loss: 0.4457 - accuracy: 0.7892
Epoch 87/100
102/102 [==============================] - 1s 11ms/step - loss: 0.4442 - accuracy: 0.7886
Epoch 88/100
102/102 [==============================] - 1s 11ms/step - loss: 0.4442 - accuracy: 0.7855
Epoch 89/100
102/102 [==============================] - 1s 13ms/step - loss: 0.4443 - 

# 2)Oversampling


In [30]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over['Exited'].value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [31]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [33]:
y_preds_over = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 5s 11ms/step - loss: 0.6538 - accuracy: 0.6181
Epoch 2/100
399/399 [==============================] - 5s 11ms/step - loss: 0.6170 - accuracy: 0.6633
Epoch 3/100
399/399 [==============================] - 4s 10ms/step - loss: 0.5889 - accuracy: 0.6924
Epoch 4/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5743 - accuracy: 0.7048
Epoch 5/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5570 - accuracy: 0.7167
Epoch 6/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5399 - accuracy: 0.7307
Epoch 7/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5242 - accuracy: 0.7447
Epoch 8/100
399/399 [==============================] - 4s 10ms/step - loss: 0.5125 - accuracy: 0.7505
Epoch 9/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5033 - accuracy: 0.7557
Epoch 10/100
399/399 [==============================] - 4s 10ms/step - loss: 0.494

399/399 [==============================] - 4s 10ms/step - loss: 0.4556 - accuracy: 0.7761
Epoch 81/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4547 - accuracy: 0.7770
Epoch 82/100
399/399 [==============================] - 4s 11ms/step - loss: 0.4540 - accuracy: 0.7784
Epoch 83/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4544 - accuracy: 0.7780
Epoch 84/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4536 - accuracy: 0.7798
Epoch 85/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4531 - accuracy: 0.7812
Epoch 86/100
399/399 [==============================] - 4s 11ms/step - loss: 0.4527 - accuracy: 0.7786
Epoch 87/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4542 - accuracy: 0.7782
Epoch 88/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4536 - accuracy: 0.7809
Epoch 89/100
399/399 [==============================] - 4s 11ms/step - loss: 0.4527 - 

# Method3: SMOTE


In [34]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [35]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

0    7963
1    7963
Name: Exited, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [37]:
y_preds_smote = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 3s 6ms/step - loss: 0.6635 - accuracy: 0.5963
Epoch 2/100
399/399 [==============================] - 2s 6ms/step - loss: 0.6022 - accuracy: 0.6800
Epoch 3/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5791 - accuracy: 0.6962
Epoch 4/100
399/399 [==============================] - 3s 9ms/step - loss: 0.5693 - accuracy: 0.7101
Epoch 5/100
399/399 [==============================] - 4s 10ms/step - loss: 0.5634 - accuracy: 0.7132
Epoch 6/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5585 - accuracy: 0.7155
Epoch 7/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5546 - accuracy: 0.7148
Epoch 8/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5497 - accuracy: 0.7194
Epoch 9/100
399/399 [==============================] - 4s 11ms/step - loss: 0.5465 - accuracy: 0.7206
Epoch 10/100
399/399 [==============================] - 5s 12ms/step - loss: 0.5425 - 

399/399 [==============================] - 4s 10ms/step - loss: 0.4325 - accuracy: 0.7943
Epoch 81/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4321 - accuracy: 0.7952
Epoch 82/100
399/399 [==============================] - 4s 9ms/step - loss: 0.4328 - accuracy: 0.7937
Epoch 83/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4320 - accuracy: 0.7930
Epoch 84/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4324 - accuracy: 0.7929
Epoch 85/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4318 - accuracy: 0.7953
Epoch 86/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4313 - accuracy: 0.7938
Epoch 87/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4317 - accuracy: 0.7952
Epoch 88/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4305 - accuracy: 0.7975
Epoch 89/100
399/399 [==============================] - 4s 10ms/step - loss: 0.4311 - a

## 4)Use of Ensemble with undersampling

In [38]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [39]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [41]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [42]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [43]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [44]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [45]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 2s 10ms/step - loss: 0.6854 - accuracy: 0.5501
Epoch 2/100
98/98 [==============================] - 1s 9ms/step - loss: 0.6581 - accuracy: 0.6186
Epoch 3/100
98/98 [==============================] - 1s 10ms/step - loss: 0.6444 - accuracy: 0.6384
Epoch 4/100
98/98 [==============================] - 1s 10ms/step - loss: 0.6352 - accuracy: 0.6451
Epoch 5/100
98/98 [==============================] - 1s 9ms/step - loss: 0.6284 - accuracy: 0.6522
Epoch 6/100
98/98 [==============================] - 1s 9ms/step - loss: 0.6230 - accuracy: 0.6586
Epoch 7/100
98/98 [==============================] - 1s 10ms/step - loss: 0.6163 - accuracy: 0.6640
Epoch 8/100
98/98 [==============================] - 1s 9ms/step - loss: 0.6101 - accuracy: 0.6758
Epoch 9/100
98/98 [==============================] - 1s 10ms/step - loss: 0.6036 - accuracy: 0.6816
Epoch 10/100
98/98 [==============================] - 1s 10ms/step - loss: 0.5971 - accuracy: 0.6861
Epo

98/98 [==============================] - 1s 10ms/step - loss: 0.4655 - accuracy: 0.7696
Epoch 83/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4662 - accuracy: 0.7715
Epoch 84/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4662 - accuracy: 0.7696
Epoch 85/100
98/98 [==============================] - 1s 11ms/step - loss: 0.4666 - accuracy: 0.7734
Epoch 86/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4660 - accuracy: 0.7693
Epoch 87/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4652 - accuracy: 0.7706
Epoch 88/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4674 - accuracy: 0.7709
Epoch 89/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4664 - accuracy: 0.7690
Epoch 90/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4647 - accuracy: 0.7667
Epoch 91/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4647 - accuracy: 0.7734
Epoch 92

In [46]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6912 - accuracy: 0.5190
Epoch 2/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6713 - accuracy: 0.5814
Epoch 3/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6565 - accuracy: 0.6246
Epoch 4/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6429 - accuracy: 0.6432
Epoch 5/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6307 - accuracy: 0.6461
Epoch 6/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6193 - accuracy: 0.6522
Epoch 7/100
98/98 [==============================] - 1s 6ms/step - loss: 0.6089 - accuracy: 0.6691
Epoch 8/100
98/98 [==============================] - 1s 6ms/step - loss: 0.5950 - accuracy: 0.6870
Epoch 9/100
98/98 [==============================] - 1s 7ms/step - loss: 0.5822 - accuracy: 0.7005
Epoch 10/100
98/98 [==============================] - 1s 10ms/step - loss: 0.5697 - accuracy: 0.7053
Epoch 11

98/98 [==============================] - 1s 10ms/step - loss: 0.4640 - accuracy: 0.7776
Epoch 83/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4644 - accuracy: 0.7766
Epoch 84/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4623 - accuracy: 0.7741
Epoch 85/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4645 - accuracy: 0.7718
Epoch 86/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4638 - accuracy: 0.7782
Epoch 87/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4637 - accuracy: 0.7738
Epoch 88/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4621 - accuracy: 0.7757
Epoch 89/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4626 - accuracy: 0.7795
Epoch 90/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4637 - accuracy: 0.7802
Epoch 91/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4614 - accuracy: 0.7808
Epoch 92

In [47]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - ETA: 0s - loss: 0.6610 - accuracy: 0.61 - 1s 6ms/step - loss: 0.6604 - accuracy: 0.6141
Epoch 2/100
87/87 [==============================] - 1s 6ms/step - loss: 0.6282 - accuracy: 0.6491
Epoch 3/100
87/87 [==============================] - 1s 6ms/step - loss: 0.6079 - accuracy: 0.6863
Epoch 4/100
87/87 [==============================] - 1s 8ms/step - loss: 0.5977 - accuracy: 0.6953
Epoch 5/100
87/87 [==============================] - 1s 10ms/step - loss: 0.5918 - accuracy: 0.6939
Epoch 6/100
87/87 [==============================] - 1s 10ms/step - loss: 0.5849 - accuracy: 0.6982
Epoch 7/100
87/87 [==============================] - 1s 10ms/step - loss: 0.5801 - accuracy: 0.7036
Epoch 8/100
87/87 [==============================] - 1s 10ms/step - loss: 0.5735 - accuracy: 0.7047
Epoch 9/100
87/87 [==============================] - 1s 10ms/step - loss: 0.5694 - accuracy: 0.7051
Epoch 10/100
87/87 [==============================] - 1s 10ms/

87/87 [==============================] - 1s 10ms/step - loss: 0.4448 - accuracy: 0.7939
Epoch 82/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4437 - accuracy: 0.7939
Epoch 83/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4444 - accuracy: 0.7957
Epoch 84/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4431 - accuracy: 0.7931
Epoch 85/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4439 - accuracy: 0.7921
Epoch 86/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4448 - accuracy: 0.7949
Epoch 87/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4432 - accuracy: 0.7935
Epoch 88/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4412 - accuracy: 0.7946
Epoch 89/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4416 - accuracy: 0.7935
Epoch 90/100
87/87 [==============================] - 1s 10ms/step - loss: 0.4416 - accuracy: 0.7917
Epo

In [48]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [49]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.93      0.75      0.83      1593
           1       0.44      0.78      0.56       407

    accuracy                           0.75      2000
   macro avg       0.69      0.76      0.70      2000
weighted avg       0.83      0.75      0.78      2000

